In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
from pycocotools.coco import *
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from matplotlib.pyplot import imsave
from matplotlib.pyplot import cm
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
dataDir='/media/D/DataSets/coco'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(annFile)
print(coco)

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
allcatIds=coco.getCatIds()
print(allcatIds)
print(len(cats))
# print(cats)
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [ ]:
#get all catIds
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print(len(nms))
catIds = coco.getCatIds(catNms=nms);
print(catIds)
for i in catIds:
    imgIds.extend(coco.getImgIds(catIds=i ))
print(len(imgIds))
unique_imgIds=set(imgIds)
print(len(unique_imgIds))

imgIds=coco.getImgIds()
print(len(imgIds))
# f=open('coco_train2017_img.txt','w')
# f.write('\n'.join(['{0:0>12}'.format(Ids) for Ids in imgIds]))
# f.close()

In [ ]:
VOC_classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
             'diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor'];
common_class=[cls for cls in VOC_classes if cls not in nms]
print(len(common_class))
print(common_class)

In [ ]:
#PASCAL VOC classes
# VOC_classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
#              'diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor'];
VOC_classes=['airplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
             'dining table','dog','horse','motorcycle','person','potted plant','sheep','couch','train','tv'];
common_class=[cls for cls in VOC_classes if cls in nms]
print(len(common_class))
print(common_class)
# catIds = coco.getCatIds(catNms=common_class);
catIds = [coco.getCatIds(catNms=cls)[0] for cls in VOC_classes];
print(catIds)
imgIds = []
for i in catIds:
    imgIds.extend(coco.getImgIds(catIds=i ))
print(len(imgIds))
unique_imgIds=set(imgIds)
print(len(unique_imgIds))
# f=open('coco_voc_pretrain_img.txt','w')
# f.write('\n'.join(['{0:0>12}'.format(Ids) for Ids in unique_imgIds]))
# f.close()

In [ ]:
select_imgIds=[]
for i in unique_imgIds:
    area_all=0
    annIds = coco.getAnnIds(imgIds=i, catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    area_list=[ann['area'] for ann in anns if ann['area']>1000]
#     area_large= [area_l for area_l in area
#     for area_i in area_list:
#     for area_i in area_list:
#         area_all+= area_i
    if len(area_list)==len(anns):
#         print(len(anns))
        select_imgIds.append(i)    
print(len(select_imgIds))
# f=open('coco_voc_pretraining_img.txt','w')
# f.write('\n'.join(['{0:0>12}'.format(Ids) for Ids in select_imgIds]))
# f.close()
f=open('coco_voc_pretraining_img_Id'+dataType+'.txt','w')
f.write('\n'.join([ str(Ids) for Ids in select_imgIds]))
f.close()

In [ ]:
pretraining_list = open('coco_voc_pretraining_img_Id'+dataType+'.txt','r')
imgIds = pretraining_list.readlines()
imgIds = [int(id) for id in imgIds]
print(len(imgIds))
pretraining_list.close   
# bnmask = coco.annToMask(anns[1])
# print(bnmask)
# plt.imshow(bnmask)
# bnmask.max()

VOC_classes=['airplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
             'dining table','dog','horse','motorcycle','person','potted plant','sheep','couch','train','tv'];
common_class=[cls for cls in VOC_classes if cls in nms]
print(len(common_class))
print(common_class)
catIds = [coco.getCatIds(catNms=cls)[0] for cls in VOC_classes];
print(catIds)
catIdIndex =[catIds.index(catId)+1 for catId in catIds]
print(catIdIndex)
count = 0
from PIL import Image
for imgId in imgIds:
    count=count+1
    print(count)
#     print(imgId)
#     print(catIds)
    img = coco.loadImgs(ids=[int(imgId)])[0]
#     print(img)
    imsize = (img["height"],img["width"])
#     print(imsize)
    bnmask=np.zeros(imsize)
    annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
#     print(annIds)
    anns = coco.loadAnns(annIds)
    ann_sel = [ann for ann in anns if ann['category_id'] in catIds]
#     print(anns)
    for ann in ann_sel:
        voc_label = catIds.index(ann['category_id'])+1
#         print(voc_label)
#         print(VOC_classes[voc_label-1])
        bnmask += coco.annToMask(ann)*voc_label
#         print(bnmask)
#         plt.imshow(bnmask);plt.axis('off')
#         print(bnmask.max())
    immask = np.asarray(bnmask)
#     print(immask.ndim)
#     print('%s/%s/%s.jpg'%(dataDir,dataType,img['file_name'].split('.')[0]))
    maskimg=Image.fromarray(immask.astype(np.uint8))
    saveDir='coco_voc_pretraining_'+dataType+'_png'
    if not os.path.exists(saveDir):
        os.mkdir(saveDir)
    maskimg.save(saveDir+'/%s.png'%(img['file_name'].split('.')[0]))
#     if immask.ndim == 3:
#         # if 3D, show as RGB
#         imsave('coco_voc_pretraining_img/%s.png'%(img['file_name'].split('.')[0]), immask, format="png")
#     else:
#         # if 2D, show as grayscale
#         imsave('coco_voc_pretraining_img/%s.png'%(img['file_name'].split('.')[0]), immask, format="png", cmap=cm.gray)

    
 


In [ ]:
annIds = coco.getAnnIds(imgIds=int(imgId), catIds=[5, 2, 16, 9, 44, 6, 3, 17, 62, 21, 67, 18, 19, 4, 1, 64, 20, 63, 7, 72], iscrowd=None)
print(annIds)